In [ ]:
!pip install geemap &> install.log

In [ ]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
!pip install geopandas

In [4]:
import ee
import json
import datetime
import pandas as pd
#import geemap
import os
import math
import geopandas as gpd
import re
import numpy as np

In [24]:
try:
    ee.Initialize()
except:
    ee.Authenticate(auth_mode='notebook')
    ee.Initialize(project='potential-evap')

In [6]:
def longest_numeric_substring(string):
    return max(re.findall(r'\d+', string), key=len)

In [39]:
ET_ts_path = 'PET_TS/'
ET_ts_exported = []
for file in os.listdir(ET_ts_path):
  file_path = os.path.join(ET_ts_path,file)
  ET_ts_exported.append(file_path[-11:-4])

print(len(ET_ts_exported))

8884


In [40]:
shapefiles_list = []
path = "../Shapefiles/GAGES_smooth_shp"
count_tot_shps = 0
count_shps_not_exp = 0

for file in os.listdir(path):
  if file.endswith('.shp'):
    count_tot_shps += 1
    file_path = os.path.join(path,file)
    basin_no = file_path[-11:-4] #string
    if (basin_no not in ET_ts_exported):
      count_shps_not_exp += 1
      shapefiles_list.append(file_path)

print(len(shapefiles_list))

0


In [37]:
bounds = None
def region_et(image):
    # reduction function
    et = image.reduceRegion( #zonal statistics
        reducer = ee.Reducer.mean(),
        geometry = bounds,
        scale = 1000 # 1 km
    )
    # set the result as a metadata property in the image
    return image.set(et)

In [38]:
for i in range(len(shapefiles_list)):
# for i in range(1):
  shapefile = gpd.read_file(shapefiles_list[i])

  features = []
  for j in range(shapefile.shape[0]):
    geom = shapefile.iloc[j:j+1,:]
    jsonDict = eval(geom.to_json())
    geojsonDict = jsonDict['features'][0]
    features.append(ee.Feature(geojsonDict))

  fc = ee.FeatureCollection(features)
  bounds = fc.geometry().bounds()

  mod16 = ee.ImageCollection("MODIS/061/MOD16A2GF").select("PET").filterBounds(bounds)
  # apply the function and filter for images that were not all masked
  ET_region = mod16.map(region_et).filter(ee.Filter.neq("PET",None))

  timeseries = ET_region.aggregate_array("PET").getInfo()
  timestamp = ET_region.aggregate_array("system:time_start").getInfo()
  dates = pd.to_datetime(np.array(timestamp)*1e6)
  region_series = pd.Series(timeseries,index=dates,name="Region PET [kg/m^2/8day]")

  #-----------------------
  region_series_df = pd.DataFrame(region_series)

  # df.plot(figsize=(15,11))
  file_PathName = 'PET_TS/' + 'basin_' + longest_numeric_substring(shapefiles_list[i]) + '.csv'
  print(i, '.) ', shapefiles_list[i][-11:-4])
  region_series_df.to_csv(path_or_buf=file_PathName,index=True)

0 .)  6754500
1 .)  7099400
2 .)  3353120
3 .)  2465292
4 .)  8201500
5 .)  7363500
6 .)  2492500
7 .)  9481500
8 .)  3519500
9 .)  3248500
10 .)  6467600
11 .)  9485000
12 .)  6336600
13 .)  7367000
14 .)  6746110
15 .)  1141000
16 .)  7196900
17 .)  3230450
18 .)  4065300
19 .)  2162093
20 .)  8023080
21 .)  4252500
22 .)  1276600
23 .)  4308500
24 .)  5030000
25 .)  6838000
26 .)  4256000
27 .)  1589795
28 .)  2153500
29 .)  1526250
30 .)  3335500
31 .)  6078500
32 .)  6729500
33 .)  3460000
34 .)  3182500
35 .)  2294781
36 .)  7301420
37 .)  2053500
38 .)  3564500
39 .)  6664400
40 .)  6211600
41 .)  3235500
42 .)  4216418
43 .)  6903900
44 .)  1356190
45 .)  3560000
46 .)  0322150
47 .)  3231000
48 .)  3297330
49 .)  2022500
50 .)  3289200
51 .)  1449360
52 .)  8111310
53 .)  7195400
54 .)  2026000
55 .)  5426250
56 .)  8073700
57 .)  6758500
58 .)  2035400
59 .)  9295100
60 .)  9211200
61 .)  2093877
62 .)  1636242
63 .)  2334480
64 .)  0256000
65 .)  1643805
66 .)  9401260
67 .)

/var/folders/59/d0zvy9t11r7gcr87xn39xwy00000gn/T/ipykernel_23178/4214465045.py:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  region_series = pd.Series(timeseries,index=dates,name="Region PET [kg/m^2/8day]")


1830 .)  8156910
1831 .)  1480300
1832 .)  2162350
1833 .)  4146500
1834 .)  1589330
1835 .)  1085500
1836 .)  1567000
1837 .)  0147800
1838 .)  2413875
1839 .)  5319500
1840 .)  1081000
1841 .)  8123800
1842 .)  7381450
1843 .)  1563500
1844 .)  5516000
1845 .)  4021000
1846 .)  2274005
1847 .)  1185500
1848 .)  1478120
1849 .)  2364570
1850 .)  7055875
1851 .)  1332500
1852 .)  1463500
1853 .)  1973269
1854 .)  3605078
1855 .)  3275300
1856 .)  9237500
1857 .)  8051500
1858 .)  4201340
1859 .)  3118500
1860 .)  3604400
1861 .)  3046000
1862 .)  7280000
1863 .)  8055000
1864 .)  9337500
1865 .)  9184000
1866 .)  6351000
1867 .)  9513835
1868 .)  5420680
1869 .)  5484650
1870 .)  3142500
1871 .)  8151500
1872 .)  9466500
1873 .)  6923250
1874 .)  3336645
1875 .)  6355500
1876 .)  7068510
1877 .)  5540160
1878 .)  2117600
1879 .)  6279940
1880 .)  6449400
1881 .)  1362950
1882 .)  1444500
1883 .)  4106000
1884 .)  2234384
1885 .)  9424900
1886 .)  1140585
1887 .)  5435500
1888 .)  41025

/var/folders/59/d0zvy9t11r7gcr87xn39xwy00000gn/T/ipykernel_23178/4214465045.py:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  region_series = pd.Series(timeseries,index=dates,name="Region PET [kg/m^2/8day]")


2473 .)  8157000
2474 .)  8083245
2475 .)  3068800
2476 .)  1565000
2477 .)  7026040
2478 .)  1083000
2479 .)  1334000
2480 .)  7287150
2481 .)  8164390
2482 .)  4079000
2483 .)  1330500
2484 .)  3361650
2485 .)  4207200
2486 .)  7081200
2487 .)  6748600
2488 .)  2131309
2489 .)  7048000
2490 .)  2298123
2491 .)  2025700
2492 .)  2301500
2493 .)  6267400
2494 .)  6099000
2495 .)  6119600
2496 .)  7336200
2497 .)  7185700
2498 .)  8394024
2499 .)  2097280
2500 .)  6882510
2501 .)  5362000
2502 .)  3431599
2503 .)  5433000
2504 .)  1518000
2505 .)  4100500
2506 .)  5437500
2507 .)  3298550
2508 .)  1349000
2509 .)  5533000
2510 .)  1293600
2511 .)  3068501
2512 .)  6890100
2513 .)  2196820
2514 .)  9415460
2515 .)  5051300
2516 .)  5537500
2517 .)  5487980
2518 .)  1054200
2519 .)  5454500
2520 .)  6921720
2521 .)  9378170
2522 .)  1099500
2523 .)  6935755
2524 .)  8068780
2525 .)  5554500
2526 .)  4063500
2527 .)  2142654
2528 .)  3310199
2529 .)  2366996
2530 .)  1274500
2531 .)  55500

/var/folders/59/d0zvy9t11r7gcr87xn39xwy00000gn/T/ipykernel_23178/4214465045.py:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  region_series = pd.Series(timeseries,index=dates,name="Region PET [kg/m^2/8day]")


2835 .)  2336030
2836 .)  7172000
2837 .)  9276000
2838 .)  8010000
2839 .)  3194700
2840 .)  7062575
2841 .)  2294491
2842 .)  7072000
2843 .)  8014500
2844 .)  8059400
2845 .)  3159000
2846 .)  4289000
2847 .)  4077400
2848 .)  5302000
2849 .)  1377500
2850 .)  5435935
2851 .)  1426500
2852 .)  6295250
2853 .)  6784800
2854 .)  4240200
2855 .)  2110701
2856 .)  2084160
2857 .)  2455900
2858 .)  1422000
2859 .)  5202000
2860 .)  1153700
2861 .)  1435100
2862 .)  5311400
2863 .)  1089100
2864 .)  2105000
2865 .)  7316500
2866 .)  9112500
2867 .)  6174000
2868 .)  6445980
2869 .)  2101500
2870 .)  6625000
2871 .)  6501200
2872 .)  7325860
2873 .)  1109070
2874 .)  7216500
2875 .)  7188838
2876 .)  3339000
2877 .)  3055250
2878 .)  1646500
2879 .)  3018350
2880 .)  7247015
2881 .)  7195855
2882 .)  6930000
2883 .)  1097300
2884 .)  1466200
2885 .)  9486800
2886 .)  9489500
2887 .)  8177700
2888 .)  2115860
2889 .)  5515080
2890 .)  2291500
2891 .)  3244000
2892 .)  9444200
2893 .)  10428

/var/folders/59/d0zvy9t11r7gcr87xn39xwy00000gn/T/ipykernel_23178/4214465045.py:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  region_series = pd.Series(timeseries,index=dates,name="Region PET [kg/m^2/8day]")


3145 .)  7010094
3146 .)  0348850
3147 .)  3056250
3148 .)  3216800
3149 .)  5065500
3150 .)  0276000
3151 .)  5587000
3152 .)  5507600
3153 .)  5061000
3154 .)  1610155
3155 .)  1271320
3156 .)  4213075
3157 .)  1581810
3158 .)  3329400
3159 .)  3478400
3160 .)  2102500
3161 .)  3435500
3162 .)  3364500
3163 .)  7170990
3164 .)  2106000
3165 .)  3431000
3166 .)  8057200
3167 .)  3360000
3168 .)  2331600
3169 .)  4211550
3170 .)  2038850
3171 .)  8329000
3172 .)  2231600
3173 .)  1421770
3174 .)  3353637
3175 .)  2165000
3176 .)  2487500
3177 .)  3303000
3178 .)  3456500
3179 .)  2161500
3180 .)  5592800
3181 .)  6036805
3182 .)  3307500
3183 .)  2483000
3184 .)  3072655
3185 .)  6465700
3186 .)  9128500
3187 .)  3556500
3188 .)  2061500
3189 .)  1414250
3190 .)  8130700
3191 .)  3358685
3192 .)  3207500
3193 .)  1148900
3194 .)  5063398
3195 .)  8382650
3196 .)  1077500
3197 .)  3375800
3198 .)  6665790
3199 .)  6038800
3200 .)  2385170
3201 .)  4260500
3202 .)  6033000
3203 .)  16738